In [11]:
from sklearn.feature_extraction.text import CountVectorizer

texts = ["blue house and blue window", "window window green house", "green house and blue blue"]

vectorizer = CountVectorizer()
term_count_matrix = vectorizer.fit_transform(texts)

# The unique words / column names
print(vectorizer.get_feature_names()) #in our sklearn version it is get_feature_names, in newer versions it's get_feature_names_out

# Each word"s amount of occurences in texts
print(term_count_matrix.toarray())



['and', 'blue', 'green', 'house', 'window']
[[1 2 0 1 1]
 [0 0 1 1 2]
 [1 2 1 1 0]]


**Looking at our data in data_cooked.csv which looks something like this (example)**


urgent contact last weekend draw show prize call claim code valid,1

get dump heap mom decid come low bore,0

ok lor soni ericsson salesman ask shuhui say quit gd use consid,0

privat account statement show un redeem point call identifi code expir,1


In [56]:
#First get the data
import numpy as np


#I took this from our prepared data and adjusted a bit to illustrate better
data= np.array([
("urgent free call expir urgent free claim free code", 1),
("heap mom bore",0),
("ok salesman quit",0),
("urgent call free identifi call free code expir urgent",1)
])

mails = data[:,0] #all rows, 0th column
labels = data[:,1] #all rows, 1st column


# Second: use the vectorizer on the mails
vectorizer = CountVectorizer()
word_count_matrix = vectorizer.fit_transform(mails)
print('Word Count Matrix looks like this:\n', word_count_matrix.toarray())




Word Count Matrix looks like this:
 [[0 1 1 1 1 3 0 0 0 0 0 0 2]
 [1 0 0 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 2 0 1 1 2 0 1 0 0 0 0 2]]


as a next step, we can start calculating:

for each word (meaning for each column): Count how often the word appears in a spam sentence and then calculate its probability as 

Occurences in Spam Mails  /  Amount all mails

In [ ]:
probabilities = {}

for index,word in enumerate(vectorizer.get_feature_names()):
    word_count_matrix[index]

